In [85]:
import pandas as pd
import numpy as np
import os

from tqdm import tqdm
from unicodedata import normalize

### Analizo Data de Tribunales y Cortes de Apelaciones

In [86]:
path_processed = "../data/processed/pjud"
path_censo = "../data/processed/censo"

In [87]:
df_regiones = pd.read_feather(f"{path_processed}/TerminoRol_feather")
df_tribunales = pd.read_feather(f"{path_processed}/ListadoTribunales_feather")
df_censo = pd.read_feather(f"{path_censo}/Censo2017_feather")

In [88]:
# Extraigo Cortes de Apelaciones asociadas a Juzgados desde este DataSet
data_cortes_apelaciones = pd.unique(df_termino_rol[['CORTE','TRIBUNAL']].values.ravel())
cortes_tribunales = []

for data in range(len(data_cortes_apelaciones)):  
    if not data_cortes_apelaciones[data].find('C.A.') == -1:
        corte_apelacion = data_cortes_apelaciones[data]
    else:
        tribunal = data_cortes_apelaciones[data]
        if tribunal.find("TRIBUNAL") != -1:
            separa_ciudad = tribunal.split("PENAL ")
        else:
            separa_ciudad = tribunal.split("GARANTIA ")
        ciudad = separa_ciudad[1]
        cortes_tribunales.append([corte_apelacion,ciudad])

In [89]:
lista_cortes = []

for trib in df_tribunales.index:
    for indice in range(len(cortes_tribunales)):
        if df_tribunales['ASIENTO'][trib] == cortes_tribunales[indice][1]:
            corte = cortes_tribunales[indice][0]
            lista_cortes.append(corte)
            break

df_tribunales['CORTE'] = lista_cortes

In [90]:
df_tribunales

,index,REGION,TRIBUNAL,ASIENTO,COMUNA,CORTE
0,0,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,HUARA,C.A. DE IQUIQUE
1,1,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,CAMIÑA,C.A. DE IQUIQUE
2,2,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,COLCHANE,C.A. DE IQUIQUE
3,3,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,IQUIQUE,C.A. DE IQUIQUE
4,4,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,POZO ALMONTE,C.A. DE IQUIQUE
...,...,...,...,...,...,...
687,341,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA PORVENIR,PORVENIR,PRIMAVERA,C.A. DE PUNTA ARENAS
688,342,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA PORVENIR,PORVENIR,TIMAUKEL,C.A. DE PUNTA ARENAS
689,343,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA CABO DE HORNOS,CABO DE HORNOS,CABO DE HORNOS,C.A. DE PUNTA ARENAS
690,344,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA CABO DE HORNOS,CABO DE HORNOS,ANTARTICA,C.A. DE PUNTA ARENAS


In [91]:
poblacion = []

for trib in df_tribunales.index:
    for indice in df_censo.index:
        if df_tribunales['COMUNA'][trib] == df_censo['NOMBRE COMUNA'][indice]:
            censado = df_censo['TOTAL POBLACIÓN EFECTIVAMENTE CENSADA'][indice]
            poblacion.append(censado)
            break

len(poblacion)

692

## CARGA DATA A ARCHIVOS FUTHER

In [76]:
# Reset el index para realizar feather
df_tribunales.reset_index(inplace = True)
# Directorio donde se guardaran archivos feather
os.makedirs(path_processed, exist_ok = True) 
df_tribunales.to_feather(f'{path_processed}/ListadoTribunalesyCortes_feather')